In [1]:
pip install simpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import simpy
import random

In [13]:
NUM_ID_CHECKERS = 7 
NUM_SCANNERS = 7

ARRIVAL_RATE = 5         
ID_CHECK_TIME = 0.75     
SCANNER_MIN = 0.5        
SCANNER_MAX = 1.0        
SIM_TIME = 480

wait_times = []

In [14]:
def passenger(env, name, id_checkers, scanners):
    arrive_time = env.now  
    print(f"{env.now:.1f}: {name} arrives")
    
    #STEP 1: Wait for ID checker
    with id_checkers.request() as request:
        yield request  # Wait in line
        print(f"{env.now:.1f}: {name} starts ID check")
        
        #ID check takes random time (exponential distribution)
        check_time = random.expovariate(1.0 / ID_CHECK_TIME)
        yield env.timeout(check_time)
        print(f"{env.now:.1f}: {name} finishes ID check")
    
    #STEP 2: Wait for scanner
    with scanners.request() as request:
        yield request  # Wait in line
        print(f"{env.now:.1f}: {name} starts scanner")
        
        #Scanner takes random time (uniform distribution)
        scan_time = random.uniform(SCANNER_MIN, SCANNER_MAX)
        yield env.timeout(scan_time)
        print(f"{env.now:.1f}: {name} finishes scanner")
    
    #Calculate total time
    total_time = env.now - arrive_time
    wait_times.append(total_time)
    print(f"{env.now:.1f}: {name} DONE! Total time: {total_time:.1f} minutes\n")

# ============================================
# PASSENGER GENERATOR - Creates passengers
# ============================================
def generate_passengers(env, id_checkers, scanners):
    """Keep creating passengers forever"""
    passenger_number = 0
    
    while True:
        #Wait for next passenger (Poisson arrival = exponential wait)
        wait = random.expovariate(ARRIVAL_RATE)
        yield env.timeout(wait)
        
        #Create new passenger
        passenger_number += 1
        env.process(passenger(env, f"Passenger_{passenger_number}", 
                              id_checkers, scanners))

In [15]:
print("🛫 AIRPORT SECURITY SIMULATION")

print(f"Configuration:")
print(f"  - ID Checkers: {NUM_ID_CHECKERS}")
print(f"  - Scanners: {NUM_SCANNERS}")
print(f"  - Arrival Rate: {ARRIVAL_RATE} passengers/minute")
print(f"  - Simulation Time: {SIM_TIME} minutes")

print()

🛫 AIRPORT SECURITY SIMULATION
Configuration:
  - ID Checkers: 7
  - Scanners: 7
  - Arrival Rate: 5 passengers/minute
  - Simulation Time: 480 minutes



In [16]:
# Create the simulation environment
env = simpy.Environment()

#Create the resources
id_checkers = simpy.Resource(env, capacity=NUM_ID_CHECKERS)
scanners = simpy.Resource(env, capacity=NUM_SCANNERS)

#Start generating passengers
env.process(generate_passengers(env, id_checkers, scanners))

#Run the simulation!
env.run(until=SIM_TIME)


0.3: Passenger_1 arrives
0.3: Passenger_1 starts ID check
0.5: Passenger_2 arrives
0.5: Passenger_2 starts ID check
0.6: Passenger_2 finishes ID check
0.6: Passenger_2 starts scanner
0.7: Passenger_1 finishes ID check
0.7: Passenger_1 starts scanner
0.9: Passenger_3 arrives
0.9: Passenger_3 starts ID check
0.9: Passenger_4 arrives
0.9: Passenger_4 starts ID check
1.4: Passenger_5 arrives
1.4: Passenger_5 starts ID check
1.4: Passenger_1 finishes scanner
1.4: Passenger_1 DONE! Total time: 1.1 minutes

1.5: Passenger_2 finishes scanner
1.5: Passenger_2 DONE! Total time: 1.0 minutes

1.6: Passenger_6 arrives
1.6: Passenger_6 starts ID check
1.6: Passenger_7 arrives
1.6: Passenger_7 starts ID check
1.7: Passenger_7 finishes ID check
1.7: Passenger_7 starts scanner
1.8: Passenger_8 arrives
1.8: Passenger_8 starts ID check
1.8: Passenger_8 finishes ID check
1.8: Passenger_8 starts scanner
1.9: Passenger_4 finishes ID check
1.9: Passenger_4 starts scanner
2.0: Passenger_9 arrives
2.0: Passeng

In [17]:
print()
print("RESULTS")
print(f"Total passengers: {len(wait_times)}")
print(f"Average wait time: {sum(wait_times)/len(wait_times):.2f} minutes")
print(f"Minimum wait time: {min(wait_times):.2f} minutes")
print(f"Maximum wait time: {max(wait_times):.2f} minutes")
print()

avg_wait = sum(wait_times) / len(wait_times)
if avg_wait < 15:
    print("SUCCESS! Average wait is under 15 minutes!")
else:
    print("FAILED! Average wait exceeds 15 minutes.")
    print(f"  Try adding more checkers or scanners!")


RESULTS
Total passengers: 2368
Average wait time: 1.53 minutes
Minimum wait time: 0.51 minutes
Maximum wait time: 8.36 minutes

SUCCESS! Average wait is under 15 minutes!
